In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# from pycaret.regression import *

from etl import ETL

plt.style.use("seaborn-v0_8")

In [ ]:
df_yield = pd.read_csv("data/barley_yield_from_1982.csv", sep=";")  # 1982 to 2018
df_climate = pd.read_parquet(
    "data/climate_data_from_1982.parquet"
)  # 1982-2014 2015-2050

In [123]:
df_yield, df_climate = ETL(df_yield, df_climate).run()

--- df_climate---
Departments/Scenario dropped because of any missing values:                                    nom_dep  scenario
date                                                
2015-01-01 12:00:00               Calvados  ssp2_4_5
2015-01-01 12:00:00            Deux_Sevres  ssp2_4_5
2015-01-01 12:00:00                Essonne  ssp2_4_5
2015-01-01 12:00:00                   Eure  ssp2_4_5
2015-01-01 12:00:00                  Rhone  ssp2_4_5
2015-01-01 12:00:00        Tarn_et_Garonne  ssp2_4_5
2015-01-01 12:00:00  Territoire_de_Belfort  ssp2_4_5
2015-01-01 12:00:00               Vaucluse  ssp2_4_5
--- df_yield ---
Departments dropped because of almost absolute absence of data:
 ['Hauts_de_Seine' 'Paris' 'Seine_SeineOise']


In [106]:
df_hist = df_yield
df_forecast = pd.DataFrame()

# Feature Creation


We will perform resample our df_climate dataset to set frequency $f$ that can be set to '1H' for hourly, '1D' for daily, '1W' for weekly, '1M' for monthly, '1Y' for yearly. We will then create new features based on the resampled data.


In [93]:
frequency = "1M"
group_col = ["scenario", "nom_dep", "code_dep", "year"]
metrics = df_climate.columns[~df_climate.columns.isin(group_col)]

## Amplitude


In [ ]:
def amplitude(group):
    return group.max() - group.min()

In [109]:
monthly_data = (
    df_climate.groupby(by=group_col).resample(frequency).agg(amplitude).reset_index()
)
new_features = ["amp_" + feat for feat in metrics]
col_rename = {old_feat: new_feat for old_feat, new_feat in zip(metrics, new_features)}
monthly_data.rename(columns=col_rename, inplace=True)
monthly_data["date"] = monthly_data["date"].apply(lambda date: date.strftime("%b"))
monthly_data

,scenario,nom_dep,code_dep,year,date,amp_daily_max_NSA_temp,amp_daily_NSA_temp,amp_precipitation
0,historical,Ain,01,1982,Jan,9.683777,8.102631,0.000293
1,historical,Ain,01,1982,Feb,14.983551,12.605316,0.000305
2,historical,Ain,01,1982,Mar,19.126007,15.886414,0.000566
3,historical,Ain,01,1982,Apr,19.172760,14.585327,0.000333
4,historical,Ain,01,1982,May,12.611084,8.893372,0.000520
...,...,...,...,...,...,...,...,...
147127,ssp5_8_5,Yvelines,78,2050,Aug,10.849060,8.088684,0.000317
147128,ssp5_8_5,Yvelines,78,2050,Sep,16.233643,13.955444,0.000193
147129,ssp5_8_5,Yvelines,78,2050,Oct,8.712646,9.545502,0.000345
147130,ssp5_8_5,Yvelines,78,2050,Nov,11.965179,9.601196,0.000220


In [ ]:
pivoted_df = monthly_data.pivot_table(
    index=["scenario", "nom_dep", "code_dep", "year"],
    columns="date",
    values=new_features,
).reset_index()
pivoted_df.columns = [
    f"{metric}_{month}".strip("_") for metric, month in pivoted_df.columns
]
pivoted_df.set_index(["year"], drop=True, inplace=True)
pivoted_df.index = pd.to_datetime(pivoted_df.index, format="%Y")
pivoted_df

,scenario,nom_dep,code_dep,amp_daily_NSA_temp_Apr,amp_daily_NSA_temp_Aug,amp_daily_NSA_temp_Dec,amp_daily_NSA_temp_Feb,amp_daily_NSA_temp_Jan,amp_daily_NSA_temp_Jul,amp_daily_NSA_temp_Jun,...,amp_precipitation_Dec,amp_precipitation_Feb,amp_precipitation_Jan,amp_precipitation_Jul,amp_precipitation_Jun,amp_precipitation_Mar,amp_precipitation_May,amp_precipitation_Nov,amp_precipitation_Oct,amp_precipitation_Sep
year,,,,,,,,,,,,,,,,,,,,,
1982-01-01,historical,Ain,01,14.585327,13.266602,14.508118,12.605316,8.102631,8.545471,9.951599,...,0.000337,0.000305,0.000293,0.000355,0.000220,0.000566,0.000520,0.000488,0.000870,0.000160
1983-01-01,historical,Ain,01,11.894348,7.150391,16.380951,17.021851,9.464539,10.034546,10.661621,...,0.000416,0.000397,0.000439,0.000694,0.000392,0.000536,0.000355,0.000363,0.000217,0.000531
1984-01-01,historical,Ain,01,14.128510,5.867737,12.289917,9.180328,15.847443,10.030670,9.929626,...,0.000269,0.000420,0.000419,0.000635,0.000335,0.000334,0.000682,0.000384,0.000519,0.000394
1985-01-01,historical,Ain,01,11.441620,8.075775,14.267517,13.399414,14.638794,6.790100,11.491058,...,0.000385,0.000473,0.000610,0.000887,0.000538,0.000358,0.000422,0.000802,0.000813,0.000430
1986-01-01,historical,Ain,01,11.271729,10.372467,9.982452,11.505005,10.697601,9.380066,12.694794,...,0.000542,0.000221,0.000214,0.000724,0.000288,0.000266,0.000588,0.000690,0.000190,0.000420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046-01-01,ssp5_8_5,Yvelines,78,10.671600,11.138885,10.903961,10.098511,6.579498,10.647217,13.361725,...,0.000108,0.000067,0.000138,0.000076,0.000130,0.000076,0.000143,0.000382,0.000298,0.000022
2047-01-01,ssp5_8_5,Yvelines,78,5.617401,10.944977,10.930328,12.897888,16.145782,16.649200,14.308533,...,0.000135,0.000169,0.000239,0.000226,0.000100,0.000119,0.000227,0.000189,0.000077,0.000192
2048-01-01,ssp5_8_5,Yvelines,78,8.661102,8.688995,10.149933,13.638367,9.829681,13.737396,11.352478,...,0.000118,0.000153,0.000218,0.000495,0.000294,0.000374,0.000173,0.000214,0.000326,0.000256


Merge to df_yield on year and department.


In [128]:
df_hist = pd.merge(
    df_hist,
    pivoted_df[pivoted_df["scenario"] == "historical"],
    left_on=["department", "year"],
    right_on=["nom_dep", "year"],
    how="inner",
)

## Mean
